# Hazelcast Python Client SQL Support Notebook

This notebook shows example usage of Hazelcast Python Client using SQL functions. Hazelcast provides in-depth SQL support for your data kept in Hazelcast clusters. You can create a mapping between a database table and your data resource and execute SQL queries on the map. 


To simplify setup, this tutorial uses Hazelcast Cloud for the cluster service. Hazelcast Cloud offers free registration with 200MB storage area. You can register the Hazelcast Cloud using following link and create a cluster in few minutes: https://cloud.hazelcast.com/. You can follow this tutorial to create a cluster in Cloud. If you have any issue, you can ask your questions in Hazelcast Community Slack channel. 

![Tux, the Linux mascot](https://media-exp1.licdn.com/dms/image/C561BAQEDzMqseo-qiA/company-background_10000/0/1626235318986?e=2147483647&v=beta&t=9QQp_S3AuK8-1nMqsSTlL0xy2_JpkNQrzFJ7D_SLUYU)


After the creation of cluster, we need to connect it with a client program. You can connect to your cluster in six programming languages including Java, .NET, Python, Go+.  To use Hazelcast, you need to have one of the client in your local computer and compile/run your program with it. Following install command downloands Hazelcast Python Client locally if you do not done it before. 

Also, we will pull over data from an API, and we will send request. So, we need requests module too.

# Install packages

In [1]:
%pip install hazelcast-python-client
%pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports

In [2]:
import hazelcast
import requests
import logging
from hazelcast.core import HazelcastJsonValue

# Connect To Hazelcast Cloud

In [3]:
logging.basicConfig(level=logging.INFO)

client = hazelcast.HazelcastClient(
    cluster_name="pr-3543",
    cloud_discovery_token="Ri5zJp6oxLlKI92DOe2YbeyXkXsB7wnOilgAjtiaoTQhGp6E30",
    statistics_enabled=True,
)

print("Connection successfull.")

INFO:hazelcast.lifecycle:HazelcastClient 5.1 is STARTING
INFO:hazelcast.lifecycle:HazelcastClient 5.1 is STARTED
INFO:hazelcast.connection:Trying to connect to Address(host=10.9.210.219, port=30116)
INFO:hazelcast.lifecycle:HazelcastClient 5.1 is CONNECTED
INFO:hazelcast.connection:Authenticated with server Address(host=10.9.210.219, port=30116):78539aa6-5a07-44c6-9417-454f65d5c9d5, server version: 5.1.1, local address: Address(host=172.28.0.2, port=50200)
INFO:hazelcast.cluster:

Members [1] {
	Member [10.9.210.219]:30116 - 78539aa6-5a07-44c6-9417-454f65d5c9d5
}

INFO:hazelcast.statistics:Client statistics enabled with the period of 3.0 seconds.
INFO:hazelcast.client:Client started


Connection successfull.


# Create Mapping between Map and Table

In [4]:
prices = client.get_map("prices").blocking()

In [5]:
def create_mapping(hz_client):
    # see: https://docs.hazelcast.com/hazelcast/5.0/sql/mapping-to-maps#json-objects
    print("Creating mapping for countries...")

    mapping_query = """
        CREATE OR REPLACE MAPPING prices (
            __key VARCHAR, 
            symbol VARCHAR,
            priceChange DOUBLE,
            priceChangePercent DECIMAL,
            weightedAvgPrice DECIMAL,
            prevClosePrice DECIMAL,
            lastPrice DECIMAL,
            lastQty DECIMAL,
            bidPrice DECIMAL,
            bidQty DECIMAL,
            askPrice DECIMAL,
            askQty DECIMAL,
            openPrice DECIMAL,
            highPrice DECIMAL,
            lowPrice DECIMAL,
            volume VARCHAR,
            quoteVolume VARCHAR,
            openTime BIGINT,
            closeTime BIGINT,
            firstId BIGINT,
            lastId BIGINT
        )
        TYPE IMap OPTIONS(
            'keyFormat' = 'varchar',
            'valueFormat' = 'json-flat'
        )
    """
    hz_client.sql.execute(mapping_query).result()
    print("Mapping for countries has been created.")
    print("--------------------")


In [6]:
create_mapping(client)

Creating mapping for countries...
Mapping for countries has been created.
--------------------


# Load Data from API

In [7]:
def load_rates(prices):
    data = requests.get('https://www.binance.com/api/v3/ticker/24hr').json()
    
    i = 0;
    for f in data:
        prices.put(str(i), HazelcastJsonValue(f))
        i += 1
            
    print("Data has been loaded.")

In [8]:
load_rates(prices)

Data has been loaded.


In [9]:
def load_exchange_info(prices):
    data = requests.get('https://www.binance.com/api/v3/ticker/24hr').json()
  
    i = 0;
    for f in data['symbols']:
        prices.put(str(i), HazelcastJsonValue(f))
        i += 1
            
    print("Data has been loaded.")

In [ ]:
load_exchange_info(prices)

# Query 